# Custom Cascade Object Detector

This notebook allow you to create a haar cascade classifier using opencv library. A virtual environment is required to simplify the use because a specific version of opencv(<4) is needed. 

See opencv cascade classifier [documentation](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html) and [tutorial](https://docs.opencv.org/3.4/dc/d88/tutorial_traincascade.html) for more information.

## Virtual Environment 
Creation virtual environment and install package to use jupyter notebook:
```console
python -m venv env
source ./env/bin/activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --user --name=env
```

Then one the jupyter notebook select the correct kernel. Be sure you are using this environment.

## Create Folders

In [1]:
import os
DIR_PATH = os.getcwd()

In [2]:
paths = {
    'DATA_PATH' : os.path.join(DIR_PATH,'data'),
    'DOWNLOAD_IMAGES_PATH' : os.path.join(DIR_PATH,'data','images','download'),
    'MODELS_PATH' : os.path.join(DIR_PATH,'data','models'),
    'NEGATIVE_IMAGES_PATH' : os.path.join(DIR_PATH,'data','images','negative'),
    'POSITIVE_IMAGES_PATH' : os.path.join(DIR_PATH,'data','images','positive'),
    'RESULT_PATH' : os.path.join(DIR_PATH,'results'),
    'TEST_IMAGES_PATH' : os.path.join(DIR_PATH,'data','images','test'),
}

In [3]:
files = {
    'CASCADE_TXT' : os.path.join(paths['DATA_PATH'],'cascade.txt'),
    'FILTER_PY' : os.path.join(DIR_PATH,'filter.py'),
    'NEGATIVE_TXT' : os.path.join(paths['DATA_PATH'],'negative.txt'),
    'POSITIVE_TXT' : os.path.join(paths['DATA_PATH'],'positive.txt'),
    'POSITIVE_VEC' : os.path.join(paths['DATA_PATH'],'positive.vec'),
}

In [13]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Install Dependencies

In [ ]:
!pip install opencv-python==3.4.11.45 matplotlib bs4 requests pyqt5

## Import Packages

In [4]:
import shutil
import time
import contextlib

import requests 
from bs4 import *

import numpy as np
import matplotlib.pyplot as plt

import cv2

## Download Images

You can upload directly images or dowload from internet with the following functions
If you download from internet, you can use the file filter.py that open a GUI to save image or not

In [ ]:
def download_images(images, folder_name):
    print(f"Total {len(images)} Image Found!")
    if len(images) != 0:
        count = 0
        for image in images:
            try:
                image_link = image["data-srcset"]
            except Exception:
                try:
                    image_link = image["data-src"]
                except Exception:
                    try:
                        image_link = image["data-fallback-src"]
                    except Exception:
                        with contextlib.suppress(Exception):
                            image_link = image["src"]

            with contextlib.suppress(Exception):
                r = requests.get(image_link).content
                try:
                    r = str(r, 'utf-8')
                except UnicodeDecodeError:
                    with open(f'{folder_name}/{time.strftime("%Y%m%d_%H%M%S")}.jpg', "wb+") as f:
                        f.write(r)
                    count += 1
        print(f"Total {count} Images Downloaded Out of {len(images)}")

In [ ]:
url = ''
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    images = soup.findAll('img')
    download_images(images,folder_name=os.path.join(paths['DOWNLOAD_IMAGES_PATH']))
else :
    print('URL not valide')

In [ ]:
# filter images
command = f'python {files["FILTER_PY"]} -i {paths["DOWNLOAD_IMAGES_PATH"]} -p {paths["POSITIVE_IMAGES_PATH"]} -n {paths["NEGATIVE_IMAGES_PATH"]}'
print(command)

## Label Images

Two possibilities to label images : 
- Use opencv function
- Use yolo annotation (see [yolo](../yolo/main.ipynb)) where you can export yolo annotation into cascade annotation as cascade.txt file

## Resize image

In [5]:
def resize_image(img_path,dimension):
    img = cv2.imread(img_path)
    img = cv2.resize(img,dimension,cv2.INTER_AREA)
    cv2.imwrite(img_path,img)

In [5]:
dimension = (640,640)
folder = paths['NEGATIVE_IMAGES_PATH']

In [17]:
for file in os.listdir(folder):
    if file.endswith('.jpg'):
        img_path = os.path.join(folder,file)
        resize_image(img_path,dimension)


## Write Positive And Negative Texte File

In case you have imported the cascade label file from the yolo format, you need to run the next case to write a correct positive text file. If you have label from opencv, it is already done.
The case to create the negative text file always need to be done.

In [6]:
with open(files['CASCADE_TXT']) as f:
    labels = [line.rstrip() for line in f]
total_label = 0
with open(files['POSITIVE_TXT'],'w') as file:
    update_labels = []
    for label in labels:
        line_list = label.split(' ')
        name = ".".join(line_list[0].split(".")[:-1])
        line_list[0] = os.path.join('images','positive',f'{name}.jpg')
        n = len(line_list[1:]) // 4
        total_label += n
        new_line = [line_list[0]] + [str(n)] + line_list[1:]
        line = " ".join(new_line)
        file.write(line)
        file.write('\n')
print(f'Total samples : {total_label}')

Total samples : 1492


In [7]:
images = os.listdir(paths['NEGATIVE_IMAGES_PATH'])
with open(files['NEGATIVE_TXT'],'w') as file:
    for image in images:
        path = os.path.join(paths['NEGATIVE_IMAGES_PATH'],image)
        file.write(path)
        file.write('\n')

## Create Vector File

In [6]:
detection_width = 30
detection_heigt = 40
num_samples = 2000 # this has to be bigger than the number of tomatos you label

In [ ]:
!opencv_createsamples -info {files['POSITIVE_TXT']} -w {detection_width} -h {detection_heigt} -num {num_samples} -vec {files['POSITIVE_VEC']}

In [ ]:
!opencv_createsamples -w {detection_width} -h {detection_heigt} -vec {files['POSITIVE_VEC']}

## Train Model

In [7]:
num_positive = 1400 # need to be less than the number of sample
num_negative = 2800 
stages = 12
max_false_alarame_rate = 0.4
min_hit_rate = 0.999

In [11]:
num_model = 0

In [ ]:
model_path = os.path.join(paths['MODELS_PATH'],f'model_{num_model}')
if not os.path.exists(model_path):
    print('Model folder does not exist. Training will start from scratch.')
    if os.name == 'posix':
        !mkdir -p {model_path}
    if os.name == 'nt':
        !mkdir {model_path}
else :
    print('Model folder already exists. Training will start from previous session')

In [ ]:
command = f'opencv_traincascade -data {model_path} -vec {files["POSITIVE_VEC"]} -bg {files["NEGATIVE_TXT"]} -w {detection_width} -h {detection_heigt} -numPos {num_positive} -numNeg {num_negative} -numStages {stages} -maxFalseAlarmRate {max_false_alarame_rate} -minHitRate {min_hit_rate}'
print(command)

In [ ]:
!{command}

## Test Model

In [21]:
model_number = 0
model_path = os.path.join(paths['MODELS_PATH'],f'model_{model_number}','cascade.xml')

In [22]:
cascade = cv2.CascadeClassifier(model_path)

In [ ]:
for image in os.listdir(paths['TEST_IMAGES_PATH'])[:3]:
    
        img_path = os.path.join(paths['TEST_IMAGES_PATH'],image)
        img = cv2.imread(img_path)
        img = cv2.resize(img,dimension)
        rectangles = cascade.detectMultiScale(img,minNeighbors=50)
        for (x, y, w, h) in rectangles:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), lineType=cv2.LINE_4)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        %matplotlib inline 
        plt.imshow(img)
        plt.show()

## Live Detection

In [23]:
cap = cv2.VideoCapture(0)

prev_frame_time = 0
new_frame_time = 0

min_neighbors = 50

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.resize(frame,dimension)
    # Make detections 
    rectangles = cascade.detectMultiScale(frame,minNeighbors=min_neighbors)
    for (x, y, w, h) in rectangles:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), lineType=cv2.LINE_4)

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame, f'FPS : {fps}Hz  minNeighbors : {min_neighbors}', (2, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 0), 1, cv2.LINE_AA)
    cv2.imshow('YOLO', frame)
    if cv2.waitKey(10) & 0xFF == ord('+'):
            min_neighbors += 1
    if cv2.waitKey(10) & 0xFF == ord('-'):
        if min_neighbors>2:
            min_neighbors -= 1
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()